In [1]:
import cv2
import mediapipe as mp
import pandas as pd

# Đọc ảnh từ webcam
cap = cv2.VideoCapture(0)

# Khởi tạo thư viện mediapipe
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

lm_list = []
label = "BODYSWING"
no_of_frames = 3

def make_landmark_timestep(results):
    print(results.pose_landmarks.landmark)
    c_lm = []
    for id, lm in enumerate(results.pose_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
        c_lm.append(lm.z)
        c_lm.append(lm.visibility)
    return c_lm

def draw_landmark_on_image(mpDraw, results, img):
    # Vẽ các đường nối
    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

    # Vẽ các điểm nút
    for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = img.shape
        print(id, lm)
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(img, (cx, cy), 10, (0, 0, 255), cv2.FILLED)
    return img


while len(lm_list) <= no_of_frames:
    ret, frame = cap.read()
    if ret:
        # Nhận diện pose
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frameRGB)

        if results.pose_landmarks:
            # Ghi nhận thông số khung xương
            lm = make_landmark_timestep(results)
            lm_list.append(lm)
            # Vẽ khung xương lên ảnh
            frame = draw_landmark_on_image(mpDraw, results, frame)

        cv2.imshow("image", frame)
        if cv2.waitKey(1) == ord('q'):
            break

# Write vào file csv
df  = pd.DataFrame(lm_list)
df.to_csv(label + ".txt")
cap.release()
cv2.destroyAllWindows()

[x: 0.4817257821559906
y: 0.5372282266616821
z: -0.690658450126648
visibility: 0.9997357726097107
, x: 0.513874888420105
y: 0.47632285952568054
z: -0.6712425947189331
visibility: 0.999473512172699
, x: 0.5350255370140076
y: 0.4759507179260254
z: -0.6712334752082825
visibility: 0.9994908571243286
, x: 0.5519525408744812
y: 0.4769326448440552
z: -0.6714654564857483
visibility: 0.99947589635849
, x: 0.45839351415634155
y: 0.4798102378845215
z: -0.6449213027954102
visibility: 0.9995226860046387
, x: 0.4420319199562073
y: 0.4834153354167938
z: -0.6444439888000488
visibility: 0.9994913339614868
, x: 0.4279955327510834
y: 0.4872225522994995
z: -0.6447920799255371
visibility: 0.9994956254959106
, x: 0.5854194164276123
y: 0.5082573890686035
z: -0.45017021894454956
visibility: 0.9995858073234558
, x: 0.41880330443382263
y: 0.518940269947052
z: -0.30000177025794983
visibility: 0.9996703863143921
, x: 0.5166098475456238
y: 0.6042792797088623
z: -0.603451132774353
visibility: 0.9998270869255066
, x

In [4]:
np.array(lm_list).shape

(4, 132)

In [5]:
lm_list

[[0.4817257821559906,
  0.5372282266616821,
  -0.690658450126648,
  0.9997357726097107,
  0.513874888420105,
  0.47632285952568054,
  -0.6712425947189331,
  0.999473512172699,
  0.5350255370140076,
  0.4759507179260254,
  -0.6712334752082825,
  0.9994908571243286,
  0.5519525408744812,
  0.4769326448440552,
  -0.6714654564857483,
  0.99947589635849,
  0.45839351415634155,
  0.4798102378845215,
  -0.6449213027954102,
  0.9995226860046387,
  0.4420319199562073,
  0.4834153354167938,
  -0.6444439888000488,
  0.9994913339614868,
  0.4279955327510834,
  0.4872225522994995,
  -0.6447920799255371,
  0.9994956254959106,
  0.5854194164276123,
  0.5082573890686035,
  -0.45017021894454956,
  0.9995858073234558,
  0.41880330443382263,
  0.518940269947052,
  -0.30000177025794983,
  0.9996703863143921,
  0.5166098475456238,
  0.6042792797088623,
  -0.603451132774353,
  0.9998270869255066,
  0.4490428864955902,
  0.6072012186050415,
  -0.560775101184845,
  0.9998371601104736,
  0.7246845960617065,
  

In [9]:
print(results)

<class 'mediapipe.python.solution_base.SolutionOutputs'>


In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf


mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    #tạm thời bỏ qua mặt
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    #mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    #mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    #tạm thời bỏ qua mặt
    # Draw face connections
    """mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) """
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    """mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )"""
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #tạm thời bỏ qua mặt
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    #lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    #rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose])  


In [3]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join("MP_Data_XOA") 

# Path for video
VIDEO_PATH = os.path.join("MP_Video_Train_XOA")

# Actions that we try to detect
actions = np.array([ "abc" ])
#actions = np.array([ "aw","ee"])
#actions = np.array([ "ow","sac" ])
#actions = np.array([ "hoi","nang","nothing"])
#"aw","ee","ow","sac", "hoi","nang","nothing" 
# Thirty videos worth of data
no_sequences = 4

# Videos are going to be 30 frames in length
sequence_length = 1

# Folder start
start_folder = 0

In [4]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            os.makedirs(os.path.join(VIDEO_PATH, action))
        except:
            pass

In [5]:
import time
cap = cv2.VideoCapture(0)

# get the size of the frame
w = int(cap.get(3))
h = int (cap.get(4))

#archive file type
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            #write video
            wv_path = os.path.join(VIDEO_PATH, action, '{}.mp4'.format(str(sequence)))
            VDWT = cv2.VideoWriter (wv_path,fourcc,30,(w,h))
            for frame_num in range(sequence_length):
                t = time.time()
                # Read feed
                ret, frame = cap.read()
                if ret == False:
                    break
                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                VDWT.write(frame)
                # Break gracefully
                print('fps', 1/(time.time()-t))
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            VDWT.release()

                    
    cap.release() 
    cv2.destroyAllWindows()

fps 0.5592257697730113
fps 0.9450314976401836
fps 0.95342729314014
fps 0.9437390914795807


In [6]:
keypoints

array([ 4.62971598e-01,  5.53350270e-01, -7.97141850e-01,  9.99731362e-01,
        4.98044550e-01,  4.93874878e-01, -7.72378027e-01,  9.99511123e-01,
        5.17327845e-01,  4.94472474e-01, -7.72241950e-01,  9.99569476e-01,
        5.32929242e-01,  4.95868593e-01, -7.72434592e-01,  9.99436557e-01,
        4.43274915e-01,  4.94576812e-01, -7.53824472e-01,  9.99474525e-01,
        4.27122504e-01,  4.95213509e-01, -7.53226817e-01,  9.99513865e-01,
        4.13402021e-01,  4.95959550e-01, -7.53613353e-01,  9.99400377e-01,
        5.65762341e-01,  5.27431667e-01, -4.86351818e-01,  9.99582112e-01,
        4.03944492e-01,  5.22122502e-01, -3.87312293e-01,  9.99652743e-01,
        4.97091353e-01,  6.17458880e-01, -6.83714032e-01,  9.99821603e-01,
        4.32989597e-01,  6.14459276e-01, -6.54533982e-01,  9.99827981e-01,
        6.91417336e-01,  8.28294575e-01, -3.20465535e-01,  9.98391330e-01,
        3.06621075e-01,  7.84207225e-01, -2.35472202e-01,  9.99244511e-01,
        7.25846171e-01,  

In [8]:
keypoints.shape

(132,)